## Creating Images in Python

For this week's assignment you actually don't have to create any new images, I'll do that for you. Your job is to use
the python imaging library to detect elements of the image cased on their color. However, I think it's important that
you know images work just a bit more, so this lecture is going to focus on creating and manipulating images using PIL.


In [ ]:
# First, lets bring in a number of libraries that we'll need
import PIL
from PIL import Image
from IPython.display import display

# And lets load the image we were working with last time
file = "north_quad.jpg"
image = Image.open(file)

# Now, for this lecture I need a smaller image, so lets resize this one. There are two
# different functions we can use, resize() and thumbnail(). They differ in subtle ways,
# thumbnail takes a single argument as a tuple (width, height), and it modifies the
# image in place and preserves the aspect ratio. Resize, on the other hand, returns a
# a new image and you have complete control over whether you want to preserve the aspect
# ratio or not. If we want to preserve the aspect ratio and use the resize method then
# we want to divide the original width and height by some shared number.
small = image.resize((image.width // 3, image.height // 3))
print(small.size)
display(small)

Great, we have a smaller version of our image now, and our original image is untouched. In a previous lecture I showed
you how an image can be changed from RGB colorspace to L colorspace, effectively creating a black and white image from a
color one. We're going to do this again, but this time we're going to make a **contact sheet**. A contact sheet is sort
of an old school method of developing film where you develop every exposure in a small thumbnail on a single piece of
photo paper. This gives you a single sheet that you can use to see all of the photos you are working with. In this
example, I want to create a contact sheet that shows the original image as well as three different greyscale images
which use the red, green, and blue channels as appropriate.


In [ ]:
# There are a few ways which we can do this, but I am going to start by creating the
# four images. We already have the base color image done, and we know from the previous lecture
# how to create the others. I'm going to store all of these images in a dictionary.

# We store our base full color image first
images = {"base": small}

# Then we will store our individual color channel versions by band (channel) name
for band_name in small.getbands():
    images[band_name] = small.getchannel(band_name)

# Now let's display some diagnostic information
print(images.keys())
for image in images.values():
    display(image)

In [ ]:
# Ok, this looks good. Lets make sure we label these images though, so no
# one gets confused. And lets put them together into one larger image in a 2x2 grid

# To put the images together into one larger image, I'll first create a new image,
# I'll call it contact_sheet, which is big enough to hold all of the smaller images.
# We do this with the new() function, and it requires us to provide a mode (we'll use
# RGB from the base image) as well as a width and height
contact_sheet = Image.new(
    images["base"].mode, (images["base"].width * 2, images["base"].height * 2)
)

# Now we loop over all of the images and place them into the contact sheet. We'll
# use the paste() method to place one image into (really, on top of) another. This
# method takes two arguments, the image to paste and the upper left hand point --
# called the origin -- where we want to paste the image.
box = (0, 0)  # the top left corner

for image in images.values():
    # Paste the image in the contact sheet
    contact_sheet.paste(image, box)

    # Let's print out some diagnostic information
    print(box)

    # Update the location for the next image. I'll use some simple logic here, and
    # will move to the right if our left was 0, and move down and to the left if
    # our left was not 0
    if box[0] == 0:
        box = (images["base"].width, box[1])
    else:
        box = (box[1], images["base"].height)

display(contact_sheet)

Here's a good little exercise for you -- take a look at our logic here, it works fine if we are creating a 2x2 image,
but what would we need to change to have it work with an arbitrary number of images? For instance, lets say we wanted to
have this as a 3x3 image, so a total of eight variations and one original. How would you modify the code? Better yet,
instead of guessing, why not try it, put a few more images in the dictionary and see if you can create a 3x3 contact
sheet.


Ok, now, let's briefly talk about text. The python imaging library allows you to render text onto an image as well, and
this is where the difference between raster images and vector images becomes really apparent. In a vector image the text
is stored as a set of drawing primitives, such as a location on the image, the size and font of the text, and the words
to render. This is then turned into a series of shapes, and when you resize the image the text remains legible and
clear. In a raster (bitmap) image, the text is converted into pixels. This means that when it is resized there isn't
much that the machine can do to ensure it remains legible, and you will often get that "pixelated" look to text in
images which have been resized. The best approach when working with raster graphics and text is add your text after you
have finished the rest of the image manipulation.


In [ ]:
# Since we already have the contact sheet created I'm going to use it as our base image
# and put the text on top of it. Like other drawing functions, text drawing is done in
# the ImageDraw module, but we also need to include ImageFont for loading system fonts.

from PIL import ImageDraw, ImageFont

# When we create a font object for drawing we need to indicate which font we will use,
# and I've arbitrarily chosen to use Roboto-Bold which is a TrueType font, and we need
# to indicate the size we want the font to be.
font = ImageFont.truetype("assets/roboto_font/Roboto-Bold.ttf", 25)

# The pattern I will use is similar to the previous example, I'm going to iterate through
# the series of images but instead of using them directly I'll just write the text information
# to the contact sheet
box = (0, 0)
for image_label in images.keys():
    # I'm going to write the text on an image with a black background, with the width
    # of the image being the size of our individual images, and the height being 30 pixels

    # Create the base image
    block = Image.new("RGB", (images["base"].width, 30), color="black")

    # Get a handle to draw on that image
    draw = ImageDraw.Draw(block)

    # Draw our text. There are two required parameters, the first parameter being the
    # "anchor" position indicating where the text will start, and the second parameter
    # being the text itself. There are numerous optional parameters, such as size and
    # font. But let's look at this "anchor" parameter. This is a two character string
    # which indicates where the text should be anchored. The PIL documentation on this
    # is great, especially if you are a font geek! But in short, "mm" means that the
    # text will appear in the middle of the anchor location, and that the text will
    # be centered both vertically and horizontally around this point. Because of this
    # we put our first anchor point in the middle of the block
    draw.text(
        (block.width // 2, block.height // 2),
        f"{image_label}",
        size=25,
        anchor="mm",
        font=font,
    )

    # See PIL documentation for more information on the anchor parameter:
    # https://pillow.readthedocs.io/en/stable/handbook/text-anchors.html#text-anchors

    # The last step is to just paste this block into the contact sheet
    contact_sheet.paste(block, box)

    # And update our location for the next iteration
    if box[0] == 0:
        box = (images["base"].width, box[1])
    else:
        box = (box[1], images["base"].height)

display(contact_sheet)

Wow, that's looking good! A contact sheet that shows our four different options, and tells us which color channel they
came from. We've created a handy function if you just want to quickly make an image black and white and want to see some
of the variations that could be formed.

Now, I could end the lecture here, but I want to show you one more thing. Remember that our base image had a color mode
of RGB -- red green and blue -- and that each of our greyscale images have a color model of L for luminosity. When you
paste those images across into the contact sheet the mode is changed is RGB. So what does this actually look like under
the hood? Let's take a look.


In [ ]:
# Let's look at a chunk of the blue greyscale image. We can do this by finding the image
# center and then subtracting the height of the bar which holds the label.
blue_image = contact_sheet.crop(
    (
        contact_sheet.width // 2,
        contact_sheet.height // 2,
        contact_sheet.width,
        contact_sheet.height,
    )
)

# Let's print out a couple of pixels in this area
print(blue_image.getpixel((0, 30)))
print(blue_image.getpixel((1, 30)))
print(blue_image.getpixel((0, 31)))
print(blue_image.getpixel((1, 31)))

So what happens underneath is that PIL modifies the image from a single channel to three channels, but does so by
repeating the pixel value across all channels. This creates an RGB-based image that looks black and white, because when
an RGB pixel has equal values in each channel, it's a shade of grey.

Ok, as you can see, I love image work, and sadly I don't get much time to do it anymore. When I was a doctoral student,
however, I did a lot of hobby work with greyscale images, including high dynamic range photography, and composite
photography which, at the time, was called gigapixel imagery. It's rewarding to be able to look at a scene through the
lens of a camera and then snap a photo, then look at the photo on your monitor and peep at individual pixels. Hopefully
you've found this series of lectures on the python imaging library interesting, and are ready to tackle this week's
assignment.
